## Create a Server on the Google Cloud Platform (GCP)

This notebook can be used to launch a CPU or GPU based server on the Google Cloud Platform(GCP).   

This assumes you have already installed the carme gcp packages and have reviewed the carme-config.  

To install the package use:
`carme package install gcp-server`

To log into the GCP CLI:
`carme cmd gcp-server login`


#Remove the comment below.     

    

### Before You Begin with GPU

You will need to upgrade your account in order to have GPU available on the Google cloud platform. increase the quota for the regions you want.



### Do you want to run the commands or just see them?  
Below we have set the carme option dryrun so that commands are printed and not executed. 

In [1]:
#To run for real, just set dryrun=''
#dryrun= ''
dryrun='--dryrun'

### Print Available Commands
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [6]:
!carme cmd gcp-server list 


carme: [INFO] These commands are currently installed:
#Start of Google Cloud Platform Commands
create_service_account: "gcloud iam service-accounts create {service_account_name}\
  \ --display-name {service_account_name}"
create_key: "gcloud iam service-accounts keys create {cwd}/gcp/{authorization_file}\
  \ --iam-account {service_account_name}@{project}.iam.gserviceaccount.com"
get_policy: "gcloud iam service-accounts get-iam-policy {service_account_name}@{project}.iam.gserviceaccount.com\
  \ --format json > {cwd}gcp/policy.json"
set_policy: "gcloud iam service-accounts set-iam-policy {service_account_name}@{project}.iam.gserviceaccount.com\
  \ {cwd}/gcp/policy.json"
login: "gcloud auth login"
create_project: "gcloud projects create {project} --set-as-default"
set_project: "gcloud config set project {project}"
set_zone: "gcloud config set compute/zone {zone}"
create_fixedip: "gcloud compute addresses create {fixedip_namespace} --region={region}"
describe_fixedip: "gcloud compute ad

### Review the Package Configuration
You should always review your configuration prior to executing any commands. 



In [7]:
!cat ../../config/gcp-server.yaml

cluster_location: gcp
email: jkuruzovich@gmail.com    #Email Associated with the account.
#Start of Google Specific Configuration
project: techfunsp2018             #Google project name
server_name: carme        #Name of your Google Cluster
region: us-east1                 #Selection from gcloud compute regions list.
zone: us-east1-b                  #Selection from gcloud compute regions list.
machine_type: n1-highmem-4       #Type of Server
boot_disk_size: 250GB
authorization_file: auth.json     #Service account authorization file.
service_account_name: carme  #Service account name.
fixedip_namespace: jupyterhub-carme
accelorator: nvidia-tesla-k80     #nvidia-tesla-p100 or nvidia-tesla-k80
image_family: ubuntu-1804-lts
image_project: image-project
gpu_count: 1


### Create Project 
Google calls them projects.  Azure calles them resource groups. Either way you need one. This useful to track spending and also ensure you delete all resources at the end. 


In [9]:
!carme cmd gcp-server 'create_project' $dryrun

carme: [INFO] Running the command: create_project
carme: [INFO] Template: gcloud projects create {project} --set-as-default
carme: [INFO] Values: gcloud projects create carme-ai --set-as-default


In [11]:
#This will set the project 
!carme cmd gcp-server set_project $dryrun

carme: [INFO] Running the command: set_project
carme: [INFO] Template: gcloud config set project {project}
carme: [INFO] Values: gcloud config set project carme-ai


In [13]:
#This will set the zone (Google Only). 
!carme cmd gcp-server set_zone $dryrun


carme: [INFO] Running the command: set_zone
carme: [INFO] Template: gcloud config set compute/zone {zone}
carme: [INFO] Values: gcloud config set compute/zone us-east1-b


### Create the Sever
This will create your server. You have to wait for a few minutes. 


In [1]:
!carme run create_gpu_server --dryrun

carme: [INFO] Running the command: create_cluster
carme: [INFO] Template: gcloud container clusters create {g_cluster_name} --num-nodes={g_num_nodes} --machine-type={g_machine_type} --zone={g_zone}
carme: [INFO] Values: gcloud container clusters create techfunsp2018v2 --num-nodes=1 --machine-type=n1-highmem-4 --zone=us-east1-b


### WAIT FOR A WHILE
This can take up to 10 minutes. 

If you get an error ".kube/config: No such file or directory" just wait, it is likely still booting up. 

### Get Credentials for Kubectl
We need to add the credentials for Kubectl to work. You need a bit of time for your Kubernetes to launch.

In [3]:
#gcloud container clusters get-credentials carme
!carme run get_credentials

carme: [INFO] Running the command: get_credentials
carme: [INFO] Template: gcloud container clusters get-credentials {g_cluster_name}
carme: [INFO] Values: gcloud container clusters get-credentials techfunsp2018v2
Executing get_credentials:
 gcloud container clusters get-credentials techfunsp2018v2
Fetching cluster endpoint and auth data.
kubeconfig entry generated for techfunsp2018v2.


In [4]:
#Check to see if we have Kubectl working. 
!kubectl cluster-info


Kubernetes master is running at https://35.229.105.53
GLBCDefaultBackend is running at https://35.229.105.53/api/v1/namespaces/kube-system/services/default-http-backend/proxy
Heapster is running at https://35.229.105.53/api/v1/namespaces/kube-system/services/heapster/proxy
KubeDNS is running at https://35.229.105.53/api/v1/namespaces/kube-system/services/kube-dns/proxy
kubernetes-dashboard is running at https://35.229.105.53/api/v1/namespaces/kube-system/services/kubernetes-dashboard/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [13]:
#Check notes with Kubectl
!kubectl get node


NAME                                           STATUS    ROLES     AGE       VERSION
gke-techfunsp2018-default-pool-a0e49821-ps26   Ready     <none>    2m        v1.8.8-gke.0


### Helm Installation.  
We are going to be utilizing Helm for  installations of a variety of analytics tools.  This command will install Tiller on your cluster.  As they say, "Happy Helming!" 

A critical factor for Helm is that you have the same version running locally and via your machine.  If you run helm version and you have the right version, then you should be fine.

```
Client: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}
```

To install the appropriate version: 

```
curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get > get_helm.sh
chmod 700 get_helm.sh
RUN get_helm.sh --version v2.6.2

```


In [5]:
#Verify Versions are equal
!carme run install_helm

carme: [INFO] Executing command block install_helm:
carme: [INFO] Running the command: install_helm1
carme: [INFO] Template: kubectl --namespace kube-system create serviceaccount tiller
carme: [INFO] Values: kubectl --namespace kube-system create serviceaccount tiller
Executing install_helm1:
 kubectl --namespace kube-system create serviceaccount tiller
serviceaccount "tiller" created
carme: [INFO] Running the command: install_helm2
carme: [INFO] Template: helm init --service-account tiller
carme: [INFO] Values: helm init --service-account tiller
Executing install_helm2:
 helm init --service-account tiller
$HELM_HOME has been configured at /Users/jasonkuruzovich/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helmi

In [2]:
!helm version


Client: &version.Version{SemVer:"v2.8.2", GitCommit:"a80231648a1473929271764b920a8e346f6de844", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.8.2", GitCommit:"a80231648a1473929271764b920a8e346f6de844", GitTreeState:"clean"}


### Secure Tiller

In [ ]:
!kubectl --namespace=kube-system patch deployment tiller-deploy --type=json --patch='[{"op": "add", "path": "/spec/template/spec/containers/0/command", "value": ["/tiller", "--listen=localhost:44134"]}]'

### Enabling Autoscaling (optional)

This should launch a pod within your kubernetes cluster that will handle autoscaling of the cluster. Note that this seems to take a while and may even timeout. Consider opening and running in a terminal session. 

In [ ]:
#Google only. Currently Not possible with azure
!carme run autoscale

In [1]:
!carme run create_fixedip

carme: [INFO] Running the command: create_fixedip
carme: [INFO] Template: gcloud compute addresses create {g_fixedip_namespace} --region={g_region}
carme: [INFO] Values: gcloud compute addresses create jupyterhub-kuberlytics --region=us-east1
Executing create_fixedip:
 gcloud compute addresses create jupyterhub-kuberlytics --region=us-east1
ERROR: (gcloud.compute.addresses.create) Could not fetch resource:
 - The resource 'projects/techfunsp2018/regions/us-east1/addresses/jupyterhub-kuberlytics' already exists



In [2]:
!carme run describe_fixedip

carme: [INFO] Running the command: describe_fixedip
carme: [INFO] Template: gcloud compute addresses describe {g_fixedip_namespace} --region={g_region}
carme: [INFO] Values: gcloud compute addresses describe jupyterhub-kuberlytics --region=us-east1
Executing describe_fixedip:
 gcloud compute addresses describe jupyterhub-kuberlytics --region=us-east1
address: 35.196.37.108
creationTimestamp: '2018-04-18T19:56:15.143-07:00'
description: ''
id: '5136955220960852896'
kind: compute#address
name: jupyterhub-kuberlytics
region: https://www.googleapis.com/compute/v1/projects/techfunsp2018/regions/us-east1
selfLink: https://www.googleapis.com/compute/v1/projects/techfunsp2018/regions/us-east1/addresses/jupyterhub-kuberlytics
status: RESERVED


In [ ]:
!carme run describe_fixedip

#### That is it! You now have your own Kubernetes cluster that is ready to go. 

### Resize a Cluster


In [7]:
#Scale the cluster 
!carme run class_size

carme: [INFO] Running the command: class_size
carme: [INFO] Template: gcloud container clusters resize {g_cluster_name} --size={g_num_nodes_class} --quiet
carme: [INFO] Values: gcloud container clusters resize techfunsp2018v2 --size=2 --quiet
Executing class_size:
 gcloud container clusters resize techfunsp2018v2 --size=2 --quiet
Resizing techfunsp2018v2...done.                                               
Updated [https://container.googleapis.com/v1/projects/techfunsp2018/zones/us-east1-b/clusters/techfunsp2018v2].


In [4]:
#Stop the cluster, effectively setting the size to 0.
!carme run stop_cluster

carme: [INFO] Running the command: stop_cluster
carme: [INFO] Template: gcloud container clusters resize {g_cluster_name} --size=0 --quiet
carme: [INFO] Values: gcloud container clusters resize techfunsp2018 --size=0 --quiet
Executing stop_cluster:
 gcloud container clusters resize techfunsp2018 --size=0 --quiet
ERROR: (gcloud.container.clusters.resize) ResponseError: code=404, message=The resource "projects/techfunsp2018/zones/us-east1-b/clusters/techfunsp2018" was not found.
No cluster named 'techfunsp2018' in techfunsp2018.


In [8]:
#Set the cluster to the normal size.
!carme run normal_size

carme: [INFO] Running the command: normal_size
carme: [INFO] Template: gcloud container clusters resize {g_cluster_name} --size={g_num_nodes} --quiet
carme: [INFO] Values: gcloud container clusters resize techfunsp2018v2 --size=1 --quiet
Executing normal_size:
 gcloud container clusters resize techfunsp2018v2 --size=1 --quiet
Resizing techfunsp2018v2...done.                                               
Updated [https://container.googleapis.com/v1/projects/techfunsp2018/zones/us-east1-b/clusters/techfunsp2018v2].


In [7]:
!gcloud container clusters resize carmecli --size=2 --quiet

Resizing carmecli...done.                                                      
Updated [https://container.googleapis.com/v1/projects/techfunsp2018/zones/us-east1-b/clusters/carmecli].


### Deleting a Kubernetes Cluster

This will delete the Kubernetes cluster by deleting the entire project. This will prefent any future charges. 

In [ ]:
#Always delete the namespace first. 
!carme run delete